In [116]:
import zipfile
import pyodbc
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
from scipy import stats
from datetime import datetime as dt, timedelta
from IPython.display import display, HTML
%matplotlib inline
pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

import functions as f

In [2]:
filename = './data/Master_Data_Union_2014_2018_B_Recency_Pre2014.txt'

In [5]:
#archive = zipfile.ZipFile('./data/Master_Data_Union_2014_2018_B_Recency_Pre2014.zip', 'r')

In [3]:
%%time
chunksize = 10**6
chunks = pd.read_csv(filename, chunksize=chunksize, sep='|',low_memory=False)
#for chunk in pd.read_csv(filename,chunksize=chunksize,sep='|',low_memory=False):
    #DF = pd.concat([DF,chunk])

CPU times: user 4.77 ms, sys: 5.09 ms, total: 9.85 ms
Wall time: 26.6 ms


In [4]:
%%time
df = chunks.get_chunk(1000000)

CPU times: user 14.9 s, sys: 6.22 s, total: 21.1 s
Wall time: 23.5 s


In [ ]:
#only selecting orders that happened between may 2017-july 2017 for sample size

In [5]:
%%time
DF_SAMPLE = df.loc[df.IS_EMAILABLE_IND=='Y']
df1 = DF_SAMPLE.copy()
df1['ORDER_DATE'] = df1.ORDER_DATE.apply(lambda x: dt.strptime(x,'%Y-%m-%d'))
#df1 = df1.loc[(df1.ORDER_DATE >= dt(2017,5,1).date()) & 
#              (df1.ORDER_DATE < dt(2017,8,1).date())]
df1 = df1.loc[(df1.ORDER_DATE.dt.month >= 5) & (df1.ORDER_DATE.dt.month < 8)]
df1 = df1[~df1.BIRTH_DATE.isnull()]

CPU times: user 13.6 s, sys: 1.19 s, total: 14.8 s
Wall time: 15.2 s


In [6]:
print 'unique customer ids:', len(df1.ILINK.unique())
print 'Number emailable:', df1.shape[0]

unique customer ids: 6538
Number emailable: 150467


In [7]:
df1['BIRTH_DATE'] = df1.BIRTH_DATE.apply(lambda x: dt.strptime(x,'%Y-%m-%d').date())
#df1['FIRST_CATALOG_DATE'] = df1.FIRST_CATALOG_DATE.apply(lambda x: dt.strptime(x,'%Y-%m-%d').date())
#df1['ORDER_DATE'] = df1.ORDER_DATE.apply(lambda x: dt.strptime(x,'%Y-%m-%d').date())
df1['FIRST_ORDER_DATE'] = df1.FIRST_ORDER_DATE.apply(lambda x: dt.strptime(x,'%Y-%m-%d').date())
#df1['FIRST_RETAIL_ORDER_DATE'] = df1.FIRST_RETAIL_ORDER_DATE.apply(lambda x: dt.strptime(x,'%Y-%m-%d').date())
#df1['FIRST_WEB_ORDER_DATE'] = df1.FIRST_WEB_ORDER_DATE.apply(lambda x: dt.strptime(x,'%Y-%m-%d').date())
#df1['JJCH_CLOSE_DATE'] = df1.JJCH_CLOSE_DATE.apply(lambda x: dt.strptime(x,'%Y-%m-%d').date())
#df1['JJCH_OPEN_DATE'] = df1.JJCH_OPEN_DATE.apply(lambda x: dt.strptime(x,'%Y-%m-%d').date())
#df1['PRIOR_ORDER_DATE'] = df1.PRIOR_ORDER_DATE.apply(lambda x: dt.strptime(x,'%m/%d/%Y').date())

In [8]:
ilinks = df1.ILINK.unique().tolist()

# Definitions

In [228]:
 def normalize(l):
    s = sum(l)
    if s == 0:
        return [0]*8
    else:
        return [i/s for i in l]


In [340]:
#make empty dataframe to add onto with customer features
columns = ['Age',
           'WovenShirts_N','KnitTops_N','Pants_N','Dresses_N',
           'WovenShirts_AvgN_Order','KnitTops_AvgN_Order','Pants_AvgN_Order','Dresses_AvgN_Order',
           'WovenShirts_Total','KnitTops_Total','Pants_Total','Dresses_Total',
           'WovenShirts_Amt_Order','KnitTops_Amt_Order','Pants_Amt_Order','Dresses_Amt_Order',
           'WovenShirts_AvgAmt','KnitTops_AvgAmt','Pants_AvgAmt','Dresses_AvgAmt']


# Exploration


In [74]:
reload(f)

<module 'functions' from 'functions.py'>

In [243]:
cols = ['Ilink','N_ws','n_ws','S_ws','s_ws','si_ws','D_ws','d_ws','di_ws','woven_shirts',
        'N_kt','n_kt','S_kt','s_kt','si_kt','D_kt','d_kt','di_kt','knit_tops',
        'N_d','n_d','S_d','s_d','si_d','D_d','d_d','di_d','dresses',
        'N_p','n_p','S_p','s_p','si_p','D_p','d_p','di_p','pants']
table = pd.DataFrame([],columns=cols)

In [244]:
%%time
#need to create rows that describe a customers freq and sales of items in specific departments
depts = ['Woven Shirts','Knit Tops','Dresses','Pants']
K = 4 #Number of Depts/topics
for ilink in ilinks[:20]:
    dfCust = df1.loc[(df1.ILINK==ilink)]
    N_u = sum([f.getTotalFreq(dfCust,dept) for dept in depts])
    fts = []
    for dept in depts:
        N = f.getTotalFreq(dfCust,dept)
        n = f.getAvgFreqOrder(dfCust,dept)
        O,Q,G,S,D = f.getTotalAmts(dfCust,dept)
        o,q,g,s,d = f.getAvgAmtsOrder(dfCust,dept)
        oi,qi,gi,si,di = f.getAvgAmtItem(dfCust,dept)
        alpha = [N,n,S,s,si,D,d,di]
        alpha = [0 if math.isnan(x) else x for x in alpha]
        alpha1 = np.array(normalize(alpha))
        sigma = (N + alpha1)/(N_u + K*alpha1)
        pref = np.dot(sigma,alpha1).round(2) + N
        #print pref
        fts.append(alpha+[pref])
    
    row = [i for sub in fts for i in sub]
    row.insert(0,ilink)
    temp = pd.DataFrame([tuple(row)],columns=cols)
    table = pd.concat([table,temp])

CPU times: user 3.07 s, sys: 22.2 ms, total: 3.09 s
Wall time: 3.09 s


In [246]:
table.sort_values('pants',ascending=False)

,Ilink,N_ws,n_ws,S_ws,s_ws,si_ws,D_ws,d_ws,di_ws,woven_shirts,N_kt,n_kt,S_kt,s_kt,si_kt,D_kt,d_kt,di_kt,knit_tops,N_d,n_d,S_d,s_d,si_d,D_d,d_d,di_d,dresses,N_p,n_p,S_p,s_p,si_p,D_p,d_p,di_p,pants
0,730,1,0.0,52.54,2.50,52.54,26.46,1.26,26.46,1.04,9,0.0,300.95,14.33,33.44,52.04,2.48,5.78,9.27,0,0.0,0.00,0.00,0,0.00,0.00,0,0,23,0.0,1052.43,50.12,45.76,418.56,19.93,18.20,23.67
0,118,12,0.0,658.99,65.90,54.92,64.97,6.50,5.41,12.32,11,1.0,389.31,38.93,35.39,86.67,8.67,7.88,11.29,0,0.0,0.00,0.00,0,0.00,0.00,0,0,14,0.0,687.66,68.77,49.12,189.33,18.93,13.52,14.37
0,726,1,0.0,55.30,6.14,55.3,23.70,2.63,23.7,1.1,3,1.0,73.89,8.21,24.63,35.09,3.90,11.70,3.25,0,0.0,0.00,0.00,0,0.00,0.00,0,0,8,3.0,340.86,37.87,42.61,75.11,8.35,9.39,8.61
0,727,17,0.0,682.13,32.48,40.13,230.79,10.99,13.58,17.47,10,0.0,265.84,12.66,26.58,116.10,5.53,11.61,10.28,0,0.0,0.00,0.00,0,0.00,0.00,0,0,8,0.0,297.69,14.18,37.21,159.26,7.58,19.91,8.23
0,503,0,0.0,0.00,0.00,0,0.00,0.00,0,0,0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1,0.0,29.84,7.46,29.84,20.15,5.04,20.15,1.14,7,1.0,372.28,93.07,53.18,78.69,19.67,11.24,7.78
0,741,0,0.0,0.00,0.00,0,0.00,0.00,0,0,9,1.0,257.42,64.36,28.60,101.00,25.25,11.22,9.51,1,0.0,39.89,9.97,39.89,20.10,5.03,20.1,1.07,7,0.0,265.29,66.32,37.90,133.65,33.41,19.09,7.40
0,88,1,1.0,49.99,8.33,49.99,0.00,0.00,0,1.08,6,1.0,211.97,35.33,35.33,0.00,0.00,0.00,6.38,2,0.0,69.00,11.50,34.5,0.00,0.00,0,2.15,6,1.0,314.99,52.50,52.50,0.00,0.00,0.00,6.38
0,212,8,0.0,396.67,56.67,49.58,250.28,35.75,31.29,8.51,2,0.0,66.49,9.50,33.25,33.50,4.79,16.75,2.14,0,0.0,0.00,0.00,0,0.00,0.00,0,0,5,2.0,303.53,43.36,60.71,83.47,11.92,16.69,5.33
0,261,3,0.0,144.38,72.19,48.13,54.60,27.30,18.2,3.36,1,0.0,62.27,31.14,62.27,6.73,3.36,6.73,1.14,0,0.0,0.00,0.00,0,0.00,0.00,0,0,4,4.0,263.14,131.57,65.78,13.85,6.92,3.46,4.46
0,4,0,0.0,0.00,0.00,0,0.00,0.00,0,0,5,3.0,154.57,51.52,30.91,83.40,27.80,16.68,5.53,0,0.0,0.00,0.00,0,0.00,0.00,0,0,4,2.0,184.58,61.53,46.15,93.40,31.13,23.35,4.43
